In [ ]:
"""
merge filtered training/validation set of SEC data 
with bankruptcy list for training/validation set

apply logic to label target as where company
that filed for bankruptcy within 1 year of filing
their annual 10-K report with SEC

"""

In [1]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql
import numpy as np
import datetime
import pickle

In [2]:
# the cleaned and filtered SEC data

with open('SEC_filtered_over100MM.pickle', 'rb') as read_file:
    filtered_over100MM = pickle.load(read_file)
    
filtered_over100MM.head()

,company_name,period_end_date,date_filed,Assets,AssetsCurrent,Liabilities,LiabilitiesCurrent,NetIncomeLoss,LiabilitiesAndStockholdersEquity,StockholdersEquity,OperatingIncomeLoss,Revenues,EarningsPerShareBasic,EarningsPerShareDiluted,Division,value_count
1,1 800 FLOWERS COM INC,2014-06-30 00:00:00.000000,2015-09-11 00:00:00.000000,267569000.0,91818000.0,81451000.0,74307000.0,NaN,267569000.0,183228000.0,23706000.0,NaN,0.24,0.23,Retail Trade,13
3,1 800 FLOWERS COM INC,2015-06-30 00:00:00.000000,2016-09-16 00:00:00.000000,497073000.0,155095000.0,286817000.0,123607000.0,NaN,497073000.0,208449000.0,37617000.0,NaN,0.31,0.30,Retail Trade,13
4,1 800 FLOWERS COM INC,2016-06-30 00:00:00.000000,2016-09-16 00:00:00.000000,506514000.0,166659000.0,263928000.0,120861000.0,NaN,506514000.0,242586000.0,43282000.0,NaN,0.57,0.55,Retail Trade,13
7,1 800 FLOWERS COM INC,2017-06-30 00:00:00.000000,2018-09-14 00:00:00.000000,552470000.0,257402000.0,270231000.0,125175000.0,NaN,552470000.0,NaN,46359000.0,1.193625e+09,0.68,0.65,Retail Trade,13
9,1 800 FLOWERS COM INC,2018-06-30 00:00:00.000000,2019-09-13 00:00:00.000000,570889000.0,273021000.0,255985000.0,124799000.0,40791000.0,570889000.0,314904000.0,41048000.0,NaN,0.63,0.61,Retail Trade,14


In [6]:
# add the labels for the target
brd_labels = pd.read_excel('debtor_list_ein_lookup.xlsx')
brd_labels.head()

,debtor_name,date_filed,chapter_filing,ein,name_in_sec_data,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,First Mariner Bancorp,2014-02-10,Chapter 11,521834860,First Mariner Bancorp,NaN,NaN,NaN,NaN
1,USEC Inc.,2014-03-05,Chapter 11,522107911,USEC INC,NaN,NaN,NaN,NaN
2,MModal Inc.,2014-03-20,Chapter 11,NaN,NaN,NaN,NaN,NaN,NaN
3,Dolan Company,2014-03-23,Chapter 11,NaN,NaN,NaN,NaN,NaN,NaN
4,"Global Geophysical Services, Inc.",2014-03-25,Chapter 11,None,GLOBAL GEOPHYSICAL SERVICES INC,NaN,NaN,NaN,NaN


In [4]:
mask = (brd_labels['date_filed'] < datetime.datetime(2020, 1, 1) ) & (brd_labels['name_in_sec_data'].notna())

labels_tv = brd_labels[mask]
labels_tv.shape

(151, 9)

In [7]:
labels_tv['FULL_NAME'] = labels_tv.loc[:,'name_in_sec_data'].str.upper()
labels_tv.head()

,debtor_name,date_filed,chapter_filing,ein,name_in_sec_data,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,FULL_NAME
0,First Mariner Bancorp,2014-02-10,Chapter 11,521834860,First Mariner Bancorp,NaN,NaN,NaN,NaN,FIRST MARINER BANCORP
1,USEC Inc.,2014-03-05,Chapter 11,522107911,USEC INC,NaN,NaN,NaN,NaN,USEC INC
4,"Global Geophysical Services, Inc.",2014-03-25,Chapter 11,None,GLOBAL GEOPHYSICAL SERVICES INC,NaN,NaN,NaN,NaN,GLOBAL GEOPHYSICAL SERVICES INC
5,James River Coal Company,2014-04-07,Chapter 11,980585280,"JAMES RIVER GROUP HOLDINGS, LTD.",NaN,NaN,NaN,NaN,"JAMES RIVER GROUP HOLDINGS, LTD."
7,Momentive Performance Materials Inc.,2014-04-13,Chapter 11,205748297,Momentive Performance Materials Inc.,NaN,NaN,NaN,NaN,MOMENTIVE PERFORMANCE MATERIALS INC.


In [8]:
brd_match_list = labels_tv[['date_filed', 'FULL_NAME']]
brd_match_list.head()

,date_filed,FULL_NAME
0,2014-02-10,FIRST MARINER BANCORP
1,2014-03-05,USEC INC
4,2014-03-25,GLOBAL GEOPHYSICAL SERVICES INC
5,2014-04-07,"JAMES RIVER GROUP HOLDINGS, LTD."
7,2014-04-13,MOMENTIVE PERFORMANCE MATERIALS INC.


In [9]:
brd_match_list.tail()

,date_filed,FULL_NAME
156,2019-10-03,EP ENERGY CORP
158,2019-10-21,NOBILIS HEALTH CORP.
159,2019-11-12,DEAN FOODS CO
160,2019-11-18,APPROACH RESOURCES INC
161,2019-12-27,"MELINTA THERAPEUTICS, INC. /NEW/"


In [10]:
brd_match_list.shape

(151, 2)

In [11]:
brd_match_list['FULL_NAME'].nunique()

143

In [13]:
# which companies are listed more than once
brd_match_list[brd_match_list.duplicated()]

,date_filed,FULL_NAME
32,2015-07-15,"WALTER ENERGY, INC."
75,2016-07-27,"ATLAS RESOURCE PARTNERS, L.P."


In [14]:
brd_match_list[brd_match_list.FULL_NAME == 'WALTER ENERGY, INC.']

,date_filed,FULL_NAME
29,2015-07-15,"WALTER ENERGY, INC."
32,2015-07-15,"WALTER ENERGY, INC."


In [15]:
brd_match_list[brd_match_list.FULL_NAME == 'ATLAS RESOURCE PARTNERS, L.P.']

,date_filed,FULL_NAME
73,2016-07-27,"ATLAS RESOURCE PARTNERS, L.P."
75,2016-07-27,"ATLAS RESOURCE PARTNERS, L.P."


In [17]:
# drop the 2 duplicated rows
brd_match_list.drop_duplicates(inplace=True)
brd_match_list.shape

<ipython-input-17-88baacd10069>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brd_match_list.drop_duplicates(inplace=True)


(149, 2)

In [18]:
# brd list was manually adjusted so that it would match company names as in the SEC data
add_brd = pd.merge(filtered_over100MM, brd_match_list, how='left', left_on='company_name', right_on='FULL_NAME')

In [19]:
add_brd.head()

,company_name,period_end_date,date_filed_x,Assets,AssetsCurrent,Liabilities,LiabilitiesCurrent,NetIncomeLoss,LiabilitiesAndStockholdersEquity,StockholdersEquity,OperatingIncomeLoss,Revenues,EarningsPerShareBasic,EarningsPerShareDiluted,Division,value_count,date_filed_y,FULL_NAME
0,1 800 FLOWERS COM INC,2014-06-30 00:00:00.000000,2015-09-11 00:00:00.000000,267569000.0,91818000.0,81451000.0,74307000.0,NaN,267569000.0,183228000.0,23706000.0,NaN,0.24,0.23,Retail Trade,13,NaT,NaN
1,1 800 FLOWERS COM INC,2015-06-30 00:00:00.000000,2016-09-16 00:00:00.000000,497073000.0,155095000.0,286817000.0,123607000.0,NaN,497073000.0,208449000.0,37617000.0,NaN,0.31,0.30,Retail Trade,13,NaT,NaN
2,1 800 FLOWERS COM INC,2016-06-30 00:00:00.000000,2016-09-16 00:00:00.000000,506514000.0,166659000.0,263928000.0,120861000.0,NaN,506514000.0,242586000.0,43282000.0,NaN,0.57,0.55,Retail Trade,13,NaT,NaN
3,1 800 FLOWERS COM INC,2017-06-30 00:00:00.000000,2018-09-14 00:00:00.000000,552470000.0,257402000.0,270231000.0,125175000.0,NaN,552470000.0,NaN,46359000.0,1.193625e+09,0.68,0.65,Retail Trade,13,NaT,NaN
4,1 800 FLOWERS COM INC,2018-06-30 00:00:00.000000,2019-09-13 00:00:00.000000,570889000.0,273021000.0,255985000.0,124799000.0,40791000.0,570889000.0,314904000.0,41048000.0,NaN,0.63,0.61,Retail Trade,14,NaT,NaN


In [23]:
# rename a couple columns

add_brd.rename(columns={'date_filed_x':'date_filed_10K', 'date_filed_y': 'date_bankruptcy'}, inplace=True)

In [29]:
add_brd[add_brd['date_bankruptcy'].notnull()].company_name.nunique()

129

In [32]:
type(add_brd['period_end_date'][0])

str

In [33]:
add_brd['period_end_date'] = pd.to_datetime(add_brd['period_end_date'])


In [48]:
## if date_bankruptcy is within 1 year of period_end_date , then target = 1

## add column that calculates difference in 2 dates

add_brd['time_delta'] = add_brd['date_bankruptcy'] - add_brd['period_end_date']

add_brd.head()

,company_name,period_end_date,date_filed_10K,Assets,AssetsCurrent,Liabilities,LiabilitiesCurrent,NetIncomeLoss,LiabilitiesAndStockholdersEquity,StockholdersEquity,...,Revenues,EarningsPerShareBasic,EarningsPerShareDiluted,Division,value_count,date_bankruptcy,FULL_NAME,time_delta,days,target
0,1 800 FLOWERS COM INC,2014-06-30,2015-09-11 00:00:00.000000,267569000.0,91818000.0,81451000.0,74307000.0,NaN,267569000.0,183228000.0,...,NaN,0.24,0.23,Retail Trade,13,NaT,NaN,NaT,NaN,0
1,1 800 FLOWERS COM INC,2015-06-30,2016-09-16 00:00:00.000000,497073000.0,155095000.0,286817000.0,123607000.0,NaN,497073000.0,208449000.0,...,NaN,0.31,0.30,Retail Trade,13,NaT,NaN,NaT,NaN,0
2,1 800 FLOWERS COM INC,2016-06-30,2016-09-16 00:00:00.000000,506514000.0,166659000.0,263928000.0,120861000.0,NaN,506514000.0,242586000.0,...,NaN,0.57,0.55,Retail Trade,13,NaT,NaN,NaT,NaN,0
3,1 800 FLOWERS COM INC,2017-06-30,2018-09-14 00:00:00.000000,552470000.0,257402000.0,270231000.0,125175000.0,NaN,552470000.0,NaN,...,1.193625e+09,0.68,0.65,Retail Trade,13,NaT,NaN,NaT,NaN,0
4,1 800 FLOWERS COM INC,2018-06-30,2019-09-13 00:00:00.000000,570889000.0,273021000.0,255985000.0,124799000.0,40791000.0,570889000.0,314904000.0,...,NaN,0.63,0.61,Retail Trade,14,NaT,NaN,NaT,NaN,0


In [49]:
add_brd['days'] = add_brd["time_delta"].dt.days

In [50]:
add_brd['target'] = np.where((add_brd['days'] >= 0) & (add_brd['days'] <=365) , 1, 0)

In [51]:
add_brd[add_brd['date_bankruptcy'].notnull()][:5]

,company_name,period_end_date,date_filed_10K,Assets,AssetsCurrent,Liabilities,LiabilitiesCurrent,NetIncomeLoss,LiabilitiesAndStockholdersEquity,StockholdersEquity,...,Revenues,EarningsPerShareBasic,EarningsPerShareDiluted,Division,value_count,date_bankruptcy,FULL_NAME,time_delta,days,target
26,"21ST CENTURY ONCOLOGY HOLDINGS, INC.",2014-03-31,2016-08-23 00:00:00.000000,1.309599e+09,192214000.0,1.397526e+09,221366000.0,-30829000.0,1.309599e+09,-148107000.0,...,NaN,NaN,NaN,Services,11,2017-05-25,"21ST CENTURY ONCOLOGY HOLDINGS, INC.",1151 days,1151.0,0
27,"21ST CENTURY ONCOLOGY HOLDINGS, INC.",2014-06-30,2016-08-23 00:00:00.000000,1.125807e+09,201092000.0,1.423081e+09,234463000.0,-210238000.0,1.125807e+09,-359017000.0,...,NaN,NaN,NaN,Services,11,2017-05-25,"21ST CENTURY ONCOLOGY HOLDINGS, INC.",1060 days,1060.0,0
28,"21ST CENTURY ONCOLOGY HOLDINGS, INC.",2014-09-30,2016-08-23 00:00:00.000000,1.197942e+09,324449000.0,1.282101e+09,231336000.0,-87883000.0,1.197942e+09,-449525000.0,...,NaN,NaN,NaN,Services,11,2017-05-25,"21ST CENTURY ONCOLOGY HOLDINGS, INC.",968 days,968.0,0
29,"21ST CENTURY ONCOLOGY HOLDINGS, INC.",2014-12-31,2016-08-23 00:00:00.000000,1.153444e+09,254548000.0,1.255775e+09,198732000.0,-192816000.0,1.153444e+09,-493439000.0,...,NaN,NaN,NaN,Services,11,2017-05-25,"21ST CENTURY ONCOLOGY HOLDINGS, INC.",876 days,876.0,0
30,"21ST CENTURY ONCOLOGY HOLDINGS, INC.",2015-03-31,2016-08-23 00:00:00.000000,1.160587e+09,252893000.0,1.276980e+09,229141000.0,-16667000.0,1.160587e+09,-530007000.0,...,NaN,NaN,NaN,Services,11,2017-05-25,"21ST CENTURY ONCOLOGY HOLDINGS, INC.",786 days,786.0,0


In [59]:
add_brd[add_brd['target'] == 1][:5]

,company_name,period_end_date,date_filed_10K,Assets,AssetsCurrent,Liabilities,LiabilitiesCurrent,NetIncomeLoss,LiabilitiesAndStockholdersEquity,StockholdersEquity,...,Revenues,EarningsPerShareBasic,EarningsPerShareDiluted,Division,value_count,date_bankruptcy,FULL_NAME,time_delta,days,target
218,ACETO CORP,2018-06-30,2018-09-28 00:00:00.000000,7.670240e+08,5.026270e+08,6.717390e+08,3.025180e+08,-1.754235e+08,7.670240e+08,9.528500e+07,...,NaN,-8.980,-4.980,Wholesale Trade,14,2019-02-19,ACETO CORP,234 days,234.0,1
423,AEROPOSTALE INC,2016-01-31,2016-05-20 00:00:00.000000,3.543830e+08,2.274060e+08,NaN,1.757150e+08,-7.929800e+07,3.543830e+08,-3.564600e+07,...,NaN,-0.995,-0.995,Retail Trade,13,2016-05-04,AEROPOSTALE INC,94 days,94.0,1
760,ALLIED NEVADA GOLD CORP.,2014-12-31,2015-03-27 00:00:00.000000,9.412380e+08,2.988470e+08,6.636800e+08,5.923760e+08,-4.900720e+08,9.412380e+08,2.775580e+08,...,NaN,-4.620,-4.620,Mining,14,2015-03-10,ALLIED NEVADA GOLD CORP.,69 days,69.0,1
810,"ALPHA NATURAL RESOURCES, INC.",2014-12-31,2015-02-26 00:00:00.000000,1.073615e+10,1.876314e+09,7.749348e+09,1.009549e+09,NaN,1.073615e+10,2.986800e+09,...,2.678846e+09,-2.250,-2.250,Mining,14,2015-08-03,"ALPHA NATURAL RESOURCES, INC.",215 days,215.0,1
827,"ALTA MESA RESOURCES, INC. /DE",2018-12-31,2019-08-27 00:00:00.000000,1.357830e+09,1.429170e+08,1.141703e+09,9.563830e+08,-1.518220e+09,1.357830e+09,-2.939300e+07,...,3.364660e+08,-8.530,-8.530,Mining,15,2019-09-11,"ALTA MESA RESOURCES, INC. /DE",254 days,254.0,1


In [53]:
add_brd.target.value_counts()

0    20395
1       98
Name: target, dtype: int64

In [55]:
add_brd.columns

Index(['company_name', 'period_end_date', 'date_filed_10K', 'Assets',
       'AssetsCurrent', 'Liabilities', 'LiabilitiesCurrent', 'NetIncomeLoss',
       'LiabilitiesAndStockholdersEquity', 'StockholdersEquity',
       'OperatingIncomeLoss', 'Revenues', 'EarningsPerShareBasic',
       'EarningsPerShareDiluted', 'Division', 'value_count', 'date_bankruptcy',
       'FULL_NAME', 'time_delta', 'days', 'target'],
      dtype='object')

In [66]:
model_data = add_brd[['company_name', 'period_end_date', 'date_bankruptcy', 
                      'days', 'Assets', 'AssetsCurrent',
                      'Liabilities', 'LiabilitiesCurrent', 'NetIncomeLoss', 
                      'LiabilitiesAndStockholdersEquity', 'StockholdersEquity', 
                      'OperatingIncomeLoss', 'Revenues', 'EarningsPerShareBasic', 
                      'EarningsPerShareDiluted', 'Division', 'target']].reset_index(drop=True)
model_data.head()

,company_name,period_end_date,date_bankruptcy,days,Assets,AssetsCurrent,Liabilities,LiabilitiesCurrent,NetIncomeLoss,LiabilitiesAndStockholdersEquity,StockholdersEquity,OperatingIncomeLoss,Revenues,EarningsPerShareBasic,EarningsPerShareDiluted,Division,target
0,1 800 FLOWERS COM INC,2014-06-30,NaT,NaN,267569000.0,91818000.0,81451000.0,74307000.0,NaN,267569000.0,183228000.0,23706000.0,NaN,0.24,0.23,Retail Trade,0
1,1 800 FLOWERS COM INC,2015-06-30,NaT,NaN,497073000.0,155095000.0,286817000.0,123607000.0,NaN,497073000.0,208449000.0,37617000.0,NaN,0.31,0.30,Retail Trade,0
2,1 800 FLOWERS COM INC,2016-06-30,NaT,NaN,506514000.0,166659000.0,263928000.0,120861000.0,NaN,506514000.0,242586000.0,43282000.0,NaN,0.57,0.55,Retail Trade,0
3,1 800 FLOWERS COM INC,2017-06-30,NaT,NaN,552470000.0,257402000.0,270231000.0,125175000.0,NaN,552470000.0,NaN,46359000.0,1.193625e+09,0.68,0.65,Retail Trade,0
4,1 800 FLOWERS COM INC,2018-06-30,NaT,NaN,570889000.0,273021000.0,255985000.0,124799000.0,40791000.0,570889000.0,314904000.0,41048000.0,NaN,0.63,0.61,Retail Trade,0


In [67]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20493 entries, 0 to 20492
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   company_name                      20493 non-null  object        
 1   period_end_date                   20493 non-null  datetime64[ns]
 2   date_bankruptcy                   463 non-null    datetime64[ns]
 3   days                              463 non-null    float64       
 4   Assets                            20493 non-null  float64       
 5   AssetsCurrent                     14219 non-null  float64       
 6   Liabilities                       16170 non-null  float64       
 7   LiabilitiesCurrent                14208 non-null  float64       
 8   NetIncomeLoss                     18584 non-null  float64       
 9   LiabilitiesAndStockholdersEquity  20150 non-null  float64       
 10  StockholdersEquity                18517 non-nu

In [68]:
# pickle the model_data df

with open('model_data.pickle', 'wb') as to_write:
    pickle.dump(model_data, to_write)